[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io/)

# Image Processing with Python

## Box Blur

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 0.1.000 | 01/11/2024 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/ImageProcessingPython/0002SciKitImageBasics.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

from numba import jit, njit

# Image Processing
import skimage as ski

# Machine Learning


# Miscellaneous
import os
from platform import python_version
import random
import timeit

# Typing
from typing import Callable, List, Tuple

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
# from bokeh.plotting import figure, show

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatRangeSlider, FloatSlider, interact, IntSlider, Layout

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

### Code Exercise

 - Single line fill

 ```python
 vallToFill = ???
 ```

 - Multi Line to Fill (At least one)

 ```python
 # You need to start writing
 ????
 ```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

???
#===============================================================#
```

In [ ]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants



In [ ]:
# Fixel Algorithms Packages


In [ ]:
# General Auxiliary Functions



## Image Filtering

Image Filtering is the set of operations which alters image pixels values based on _spatial operations_.  
The operators are usually categorized by 2 main properties:

 - Linearity: $f \left( \alpha x + \beta y \right) = \alpha f \left( x \right) + \beta f \left( y \right)$.
 - Shift Invariance: $S \left( f \left( x \right) \right) = f \left( S \left( x \right) \right)$ where $S \left( \cdot \right)$ is a _Shift Operator_. 

Filters which are both _linear_ and _shift invariant_ (LSI) are applied using [Convolution](https://en.wikipedia.org/wiki/Convolution).

Given a realization of a filter, it can be also analyzed using concepts of _Signal Processing_.  
Namely its effect on the image _Spectrum_. Specifically if its operation is one of:
 - _Low Pass Filter_ (LPF): Removal of high frequency details.
 - _High Pass Filter_ (HPF): Removal of low frequency details.

This notebooks implements a simple LSI filter, the _Box Blur_.

* <font color='brown'>(**#**)</font> An LSI filter can be applied in Frequency Domain using the [Convolution Theorem](https://en.wikipedia.org/wiki/Convolution_theorem).
* <font color='brown'>(**#**)</font> Filters might be neither LPF nor HPF.

In [ ]:
# Parameters

# imgUrl = 'https://i.imgur.com/3BbIXdH.png' #<! A turtle climbing the Everest!
imgUrl = 'https://i.postimg.cc/63rN33GZ/3BbIXdH.png' #<! A turtle climbing the Everest!

boxRadius   = 5
lBoxRadius  = [1, 3, 5, 7, 9]

## Generate Data

This section loads the image used to evaluate the results.  

In [ ]:
# Load / Generate Data

mI = ski.io.imread(imgUrl)
mI = ski.util.img_as_float64(mI)
mI = np.mean(mI, axis = 2) #<! Single channel image

In [ ]:
# Display the Data

hF, hA = plt.subplots(figsize = (8, 6))
hA.imshow(mI, cmap = 'gray')
hA.set_title(f'Input Image');

## Box Blur

Box Blur is one of the most basic, yet useful, filters in Image Processing.  
It basically maps a rectangle rolling window to its mean value.

![](https://github.com/FixelAlgorithmsTeam/FixelCourses/raw/master/ImageProcessingMethods/Assets/BoxBlur.svg)

This section implements a _Box Blur_ in 3 different implementations:
 - Using [_Integral Image_](https://en.wikipedia.org/wiki/Summed-area_table) / [_Running Sum_](https://en.wikipedia.org/wiki/Running_total).
 - Using SciKit Image's [`skimage.util.view_as_windows()`](https://scikit-image.org/docs/stable/api/skimage.util.html#skimage.util.view_as_windows).

The reference is given by SciPy's built in function [`scipy.ndimage.uniform_filter()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.uniform_filter.html).

* <font color='brown'>(**#**)</font> The blurring "quality" of the Box Blur is considered to be low (The roll off on the frequency domain).
* <font color='brown'>(**#**)</font> In some cases the output is the sum and not the mean value.
* <font color='brown'>(**#**)</font> The Box Blur is a [Separable Filter](https://en.wikipedia.org/wiki/Separable_filter).


### Implementation Using Running Sum / Integral Image

This section implements the Box Blur using either Integral Image or Running Sum.

#### Running Sum / Mean

Given a samples $\boldsymbol{x} = {\left[ 0.2, 0.7, 0.9, 0.5, 0.6 \right]}^{T}$.  
For a window of size 3, the first valid window is centered at the 2nd element: $\boldsymbol{w}_{2} = {\left[ 0.2, 0.7, 0.9 \right]}^{T}$.
The sum value is given by: ${w}_{2} = \boldsymbol{1}^{T} \boldsymbol{w}_{2}$.  
For the next window ${w}_{3} = {w}_{2} + {x}_{4} - {x}_{1}$. In general ${w}_{k} = {w}_{k - 1} + {x}_{k + r} - {w}_{k - 1 - r}$ where $r$ is the window radius.

#### Integral Image

The _Integral Image_ of $\boldsymbol{I}$ is given by:

$$ {T}_{m, n} = \sum_{i = 1, j = 1}^{m , n} {I}_{i, j} $$


* <font color='red'>(**?**)</font> Think of possible issues with the Integral Image / Summed Area Table. Specifically consider large size image and the data format.  

<!-- Images of type `UInt8` / `UInt16` / `UInt32` might overflow for large images. Using `Float16` / `Float32` and even `Float64` might cause inaccuracies. -->

In [ ]:
# Box Blur by Integral Image / Running Sum

#===========================Fill This===========================#
# 1. Implement the Box Blur as a function using either Running Sum or Integral Image.
# 2. The function parameters are `mI`, `boxRadius`.
# 3. Pad the input properly, if needed, using `np.pad()` with `mode = 'edge'`.
# !! Use the separability property of the Box Blur.

def BoxPlot( mI: np.ndarray, boxRadius: int ) -> np.ndarray:    
    """
    Quantizes a given input value into one of the specified discrete levels within a given range.

    This function maps a continuous input value (`valIn`) within the range `[valMin, valMax]` 
    to one of `numLevels` equally spaced quantization levels in that range. The output value is 
    calculated to be the nearest quantized level based on the input.

    Parameters
    ----------
    valIn : float
        The input value to be quantized, expected to be in the range `[valMin, valMax]`.
    numLevels : int
        The number of quantization levels. Must be an integer greater than 1.
    valMin : float
        The minimum value of the quantization range.
    valMax : float
        The maximum value of the quantization range.

    Returns
    -------
    valOut : float
        The quantized output value, mapped to one of the `numLevels` discrete levels within the 
        range `[valMin, valMax]`.

    Notes
    -----
    - This function assumes that `valIn` is within the specified range `[valMin, valMax]`.
      If `valIn` falls outside this range, the result might not be as expected.
    - The quantization levels are linearly spaced between `valMin` and `valMax`, with each level 
      representing a segment of the input range.

    Examples
    --------
    >>> Quantizer(0.3, 4, 0.0, 1.0)
    0.3333333333333333  # Maps 0.3 to one of the 4 quantization levels within [0.0, 1.0]

    >>> Quantizer(0.75, 8, 0.0, 1.0)
    0.7142857142857143  # Maps 0.75 to one of the 8 quantization levels within [0.0, 1.0]

    Calculation Details
    -------------------
    The quantization grid is defined by evenly spaced levels between `valMin` and `valMax`. 
    For example, with `numLevels = 4`, `valMin = 0.0`, and `valMax = 1.0`, the quantization 
    levels will correspond to values approximately equal to `[0.0, 0.333, 0.667, 1.0]`.
    """

    mP = np.pad(mI, (boxRadius, boxRadius), mode = 'edge')

    return mO

#===============================================================#

In [ ]:
# Box Blur by SciKit Image `view_as_windows()`

#===========================Fill This===========================#
# 1. Implement the Box Blur as a function using `skimage.util.view_as_windows()`.
# 2. The function parameters are `mI`, `boxRadius`.
# 3. Pad the input properly, if needed, using `np.pad()` with `mode = 'edge'`.
# !! Use the separability property of the Box Blur.

def BoxPlotView( mI: np.ndarray, boxRadius: int ) -> np.ndarray:    
    """
    Quantizes a given input value into one of the specified discrete levels within a given range.

    This function maps a continuous input value (`valIn`) within the range `[valMin, valMax]` 
    to one of `numLevels` equally spaced quantization levels in that range. The output value is 
    calculated to be the nearest quantized level based on the input.

    Parameters
    ----------
    valIn : float
        The input value to be quantized, expected to be in the range `[valMin, valMax]`.
    numLevels : int
        The number of quantization levels. Must be an integer greater than 1.
    valMin : float
        The minimum value of the quantization range.
    valMax : float
        The maximum value of the quantization range.

    Returns
    -------
    valOut : float
        The quantized output value, mapped to one of the `numLevels` discrete levels within the 
        range `[valMin, valMax]`.

    Notes
    -----
    - This function assumes that `valIn` is within the specified range `[valMin, valMax]`.
      If `valIn` falls outside this range, the result might not be as expected.
    - The quantization levels are linearly spaced between `valMin` and `valMax`, with each level 
      representing a segment of the input range.

    Examples
    --------
    >>> Quantizer(0.3, 4, 0.0, 1.0)
    0.3333333333333333  # Maps 0.3 to one of the 4 quantization levels within [0.0, 1.0]

    >>> Quantizer(0.75, 8, 0.0, 1.0)
    0.7142857142857143  # Maps 0.75 to one of the 8 quantization levels within [0.0, 1.0]

    Calculation Details
    -------------------
    The quantization grid is defined by evenly spaced levels between `valMin` and `valMax`. 
    For example, with `numLevels = 4`, `valMin = 0.0`, and `valMax = 1.0`, the quantization 
    levels will correspond to values approximately equal to `[0.0, 0.333, 0.667, 1.0]`.
    """

    mP = np.pad(mI, (boxRadius, boxRadius), mode = 'edge')
    tW = ski.util.view_as_windows(mP, (2 * boxRadius + 1, 2 * boxRadius + 1))
    mO = np.mean(tW, axis = (2, 3))

    return mO

#===============================================================#

* <font color='brown'>(**#**)</font> The function `view_as_windows()` is mainly for Non Linear and / or Shift Variant operators which can not be implemented using _Convolution_.